# 아두이노 온습도 데이터 받기

- 데이터 저장하기

In [21]:
import serial
import time
import csv

# 시리얼 포트 설정
SERIAL_PORT = 'COM4'  # 아두이노가 연결된 포트를 지정합니다 (Windows: COMx, Mac/Linux: /dev/ttyUSBx)
BAUD_RATE = 9600      # 아두이노에서 설정한 baud rate와 동일하게 설정합니다
OUTPUT_FILE = r"E:\\충남대학교\\서류\\기타\\대전영재고 실습강의\\data\\dht22_data.csv"  # 데이터를 저장할 CSV 파일 경로

def read_from_arduino():
    ser = None  # 시리얼 객체 초기화
    try:
        # 시리얼 포트 열기
        ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)  # 지정한 포트와 baud rate로 시리얼 통신 설정
        print(f"Connected to {SERIAL_PORT}")
        time.sleep(2)  # 아두이노 초기화 시간을 주기 위해 2초 대기

        # CSV 파일 열기
        with open(OUTPUT_FILE, mode='w', newline='') as file:  # 지정한 경로에 CSV 파일을 쓰기 모드로 엽니다
            writer = csv.writer(file)  # CSV 파일에 데이터를 작성하기 위한 writer 객체 생성
            writer.writerow(["Timestamp", "Temperature", "Humidity"])  # CSV 파일에 헤더 작성

            while True:  # 데이터 읽기 무한 루프
                # 아두이노에서 데이터 읽기
                line = ser.readline().decode('utf-8').strip()  # 시리얼 데이터를 읽고 UTF-8로 디코딩 후 양쪽 공백 제거
                if line:  # 데이터가 비어 있지 않을 경우 처리
                    try:
                        # 데이터를 쉼표로 구분하여 온도와 습도 값으로 파싱
                        temperature, humidity = map(float, line.split(","))
                        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")  # 현재 시간을 타임스탬프 형식으로 생성

                        # 읽은 데이터 출력
                        print(f"{timestamp} | Temperature: {temperature} °C | Humidity: {humidity} %")

                        # 데이터를 CSV 파일에 작성
                        writer.writerow([timestamp, temperature, humidity])
                        file.flush()  # 데이터를 즉시 파일에 기록
                    except ValueError:  # 데이터 형식이 잘못된 경우 예외 처리
                        print("Invalid data received, skipping...")
    except serial.SerialException as e:  # 시리얼 포트 연결 실패 예외 처리
        print(f"Could not open serial port: {e}")
    except KeyboardInterrupt:  # 사용자가 프로그램 종료 시 Ctrl+C 입력 시 예외 처리
        print("\nProgram terminated by user.")
    finally:
        if ser is not None and ser.is_open:  # 시리얼 객체가 초기화되었고 포트가 열려 있는 경우
            ser.close()  # 시리얼 포트를 닫습니다
            print("Serial connection closed.")

if __name__ == "__main__":
    read_from_arduino()  # 프로그램 실행

Connected to COM4
2025-01-14 07:58:05 | Temperature: 30.0 °C | Humidity: 30.9 %
2025-01-14 07:58:07 | Temperature: 30.0 °C | Humidity: 31.6 %
2025-01-14 07:58:09 | Temperature: 30.0 °C | Humidity: 31.9 %
2025-01-14 07:58:11 | Temperature: 30.0 °C | Humidity: 32.2 %
2025-01-14 07:58:13 | Temperature: 30.0 °C | Humidity: 32.3 %

Program terminated by user.
Serial connection closed.


- 매 정각 10초마다 저장

In [20]:
import serial
import time
import pandas as pd

# 시리얼 포트 및 설정
SERIAL_PORT = 'COM4'  # 아두이노 연결 포트
BAUD_RATE = 9600      # 아두이노 Baud Rate
EXCEL_FILE = r"E:\충남대학교\서류\기타\대전영재고 실습강의\data\dht22_data_1.xlsx"  # 저장할 엑셀 파일 이름

def wait_until_next_interval(interval=10):
    """현재 시간을 기준으로 다음 interval 초까지 대기"""
    now = time.time()
    sleep_time = interval - (now % interval)
    time.sleep(sleep_time)

def read_from_arduino():
    try:
        # 시리얼 포트 열기
        ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
        print(f"Connected to {SERIAL_PORT}")
        time.sleep(2)  # 아두이노 초기화 시간 대기

        # 데이터를 저장할 리스트 초기화
        data = []

        # 매 10초 단위 정각에 데이터 저장
        while True:
            # 대기: 다음 10초 단위로 이동
            wait_until_next_interval(interval=10)

            # 아두이노에서 데이터 읽기
            line = ser.readline().decode('utf-8').strip()

            # # 디버깅: 원시 데이터 출력
            # print(f"Raw data: {line}")

            if line:
                try:
                    # 데이터 파싱
                    temperature, humidity = map(float, line.split(","))
                    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

                    # 데이터를 리스트에 추가
                    data.append({"Timestamp": timestamp, "Temperature (°C)": temperature, "Humidity (%)": humidity})

                    # 실시간 데이터 출력
                    print(f"{timestamp} | Temperature: {temperature} °C | Humidity: {humidity} %")

                    # 데이터를 엑셀 파일에 저장
                    df = pd.DataFrame(data)
                    df.to_excel(EXCEL_FILE, index=False, engine='openpyxl')
                    print(f"Saved {len(data)} records to {EXCEL_FILE}")

                except ValueError:
                    print("Invalid data format, skipping...")
    except serial.SerialException as e:
        print(f"Serial connection error: {e}")
    except KeyboardInterrupt:
        print("\nProgram terminated by user.")
        if data:
            # 종료 시 데이터를 저장
            df = pd.DataFrame(data)
            df.to_excel(EXCEL_FILE, index=False, engine='openpyxl')
            print(f"Final data saved to {EXCEL_FILE}")
    finally:
        ser.close()
        print("Serial connection closed.")

if __name__ == "__main__":
    read_from_arduino()

Connected to COM4
2025-01-14 07:57:01 | Temperature: 30.0 °C | Humidity: 32.3 %
2025-01-14 07:57:03 | Temperature: 30.0 °C | Humidity: 33.2 %
2025-01-14 07:57:05 | Temperature: 30.0 °C | Humidity: 33.4 %
2025-01-14 07:57:07 | Temperature: 30.0 °C | Humidity: 33.7 %
2025-01-14 07:57:09 | Temperature: 30.0 °C | Humidity: 33.8 %
2025-01-14 07:57:11 | Temperature: 30.0 °C | Humidity: 33.8 %

Program terminated by user.
Serial connection closed.


- 온습도 센서 두 개 사용

In [ ]:
import serial
import time
import pandas as pd

# 시리얼 포트 및 설정
SERIAL_PORT = 'COM4'  # 아두이노 연결 포트
BAUD_RATE = 9600      # 아두이노 Baud Rate
EXCEL_FILE = r"E:\\충남대학교\\서류\\기타\\대전영재고 실습강의\\data\\dht22_data_1.xlsx"  # 저장할 엑셀 파일 이름

def wait_until_next_interval(interval=10):
    """현재 시간을 기준으로 다음 interval 초까지 대기"""
    now = time.time()
    sleep_time = interval - (now % interval)
    time.sleep(sleep_time)

def read_from_arduino():
    try:
        # 시리얼 포트 열기
        ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
        print(f"Connected to {SERIAL_PORT}")
        time.sleep(2)  # 아두이노 초기화 시간 대기

        # 데이터를 저장할 리스트 초기화
        data = []

        # 매 10초 단위 정각에 데이터 저장
        while True:
            # 대기: 다음 10초 단위로 이동
            wait_until_next_interval(interval=10)

            # 아두이노에서 데이터 읽기
            line = ser.readline().decode('utf-8').strip()

            # # 디버깅: 원시 데이터 출력
            # print(f"Raw data: {line}")

            if line:
                try:
                    # 데이터 파싱
                    values = list(map(float, line.split(",")))
                    if len(values) == 4:  # 온습도 센서 2개의 데이터 처리
                        humidity1, temperature1, humidity2, temperature2 = values
                        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

                        # 데이터를 리스트에 추가
                        data.append({
                            "Timestamp": timestamp,
                            "Temperature1 (°C)": temperature1,
                            "Humidity1 (%)": humidity1,
                            "Temperature2 (°C)": temperature2,
                            "Humidity2 (%)": humidity2
                        })

                        # 실시간 데이터 출력
                        print(f"{timestamp} | Sensor1 -> Temperature: {temperature1} °C, Humidity: {humidity1} % | Sensor2 -> Temperature: {temperature2} °C, Humidity: {humidity2} %")

                        # 데이터를 엑셀 파일에 저장
                        df = pd.DataFrame(data)
                        df.to_excel(EXCEL_FILE, index=False, engine='openpyxl')
                        print(f"Saved {len(data)} records to {EXCEL_FILE}")

                except ValueError:
                    print("Invalid data format, skipping...")
    except serial.SerialException as e:
        print(f"Serial connection error: {e}")
    except KeyboardInterrupt:
        print("\nProgram terminated by user.")
        if data:
            # 종료 시 데이터를 저장
            df = pd.DataFrame(data)
            df.to_excel(EXCEL_FILE, index=False, engine='openpyxl')
            print(f"Final data saved to {EXCEL_FILE}")
    finally:
        ser.close()
        print("Serial connection closed.")

if __name__ == "__main__":
    read_from_arduino()

- 환기팬 제어

In [ ]:
import serial
import time
import os
from datetime import datetime

# 시리얼 포트 설정 (환경에 맞게 수정)
SERIAL_PORT = "COM4"  # Linux 예시: /dev/ttyACM0, Windows 예시: COM3
BAUD_RATE = 9600

# 파일 경로 설정
data_file_path = "/home/seungun/OneDrive/room4/DHT0829.txt"
relay_file_path = "/home/seungun/OneDrive/room4/relay_status.txt"

# 파일 초기화 함수
def initialize_file(file_path, header):
    if not os.path.exists(file_path):
        with open(file_path, "w") as file:
            file.write(header + "\n")

# 데이터 파일 초기화
initialize_file(data_file_path, "날짜, 습도1, 온도1, 습도2, 온도2")
initialize_file(relay_file_path, "날짜, 릴레이 상태")

# 파일에 데이터 작성 함수
def write_to_file(data, file_path):
    with open(file_path, "a") as file:
        file.write(data)

# 시리얼 포트 연결
try:
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE)
    print(f"Connected to {SERIAL_PORT} at {BAUD_RATE} bps")
except serial.SerialException as e:
    print(f"Error: Could not connect to {SERIAL_PORT}. {e}")
    exit(1)

# 데이터 수집 루프
data_arduino = ["", "", "", ""]
try:
    while True:
        now = datetime.now()
        try:
            data = ser.readline().decode('utf-8').strip()

            if data.startswith("A,"):
                sensor_data = data[2:].split(',')
                if len(sensor_data) == 4:
                    data_arduino = sensor_data
                    formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")
                    combined_data = formatted_time + "," + ",".join(data_arduino) + "\n"
                    write_to_file(combined_data, data_file_path)
                    print(f"Recorded: {combined_data.strip()}")

            elif data.startswith("R,"):
                relay_status = data[2:]
                formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")
                relay_data = formatted_time + "," + relay_status + "\n"
                write_to_file(relay_data, relay_file_path)
                print(f"Relay status recorded: {relay_data.strip()}")

        except Exception as e:
            print(f"Error reading data: {e}")

        time.sleep(10)  # 10초 대기 후 다음 데이터 확인

except KeyboardInterrupt:
    print("Program interrupted by user.")

finally:
    if ser.is_open:
        ser.close()
        print("Serial port closed.")

In [ ]:
import serial
import time
import os
from datetime import datetime

ser = serial.Serial('/dev/ttyACM0', 9600)

data_file_path = "/home/seungun/OneDrive/room4/DHT0829.txt"
relay_file_path = "/home/seungun/OneDrive/room4/relay_status.txt"

def write_to_file(data, file_path):
    with open(file_path, "a") as file:
        file.write(data)

# 온습도 데이터 파일 초기화
if not os.path.exists(data_file_path):
    with open(data_file_path, "w") as file:
        file.write("날짜, 습도1, 온도1, 습도2, 온도2\n")

# 릴레이 상태 파일 초기화
if not os.path.exists(relay_file_path):
    with open(relay_file_path, "w") as file:
        file.write("날짜, 릴레이 상태\n")

data_arduino = ["", "", "", ""]

while True:
    try:
        now = datetime.now()
        data = ser.readline().decode('utf-8').strip()

        if data.startswith("A,"):
            sensor_data = data[2:].split(',')
            if len(sensor_data) == 4:
                data_arduino = sensor_data
                formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")
                combined_data = formatted_time + "," + ",".join(data_arduino) + "\n"
                write_to_file(combined_data, data_file_path)
                print(f"Recorded: {combined_data.strip()}")

        elif data.startswith("R,"):
            relay_status = data[2:]
            formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")
            relay_data = formatted_time + "," + relay_status + "\n"
            write_to_file(relay_data, relay_file_path)
            print(f"Relay status recorded: {relay_data.strip()}")

        time.sleep(10)  # 10초 대기 후 다음 데이터 확인

    except KeyboardInterrupt:
        print("Interrupted.")
        break
    except Exception as e:
        print("An error occurred:", e)